In [1]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import os

# ==============================================================================
# 1. CARGA DE CONFIGURAÇÃO
# ==============================================================================

def find_project_root(anchor_file="conf/config.yaml"):
    """
    Sobe os diretórios a partir do notebook atual até encontrar
    a pasta onde 'conf/config.yaml' existe.
    """
    current_path = Path.cwd()
    
    # Tenta no diretório atual e sobe até o raiz do sistema
    for parent in [current_path] + list(current_path.parents):
        potential_config = parent / anchor_file
        if potential_config.exists():
            return parent
            
    raise FileNotFoundError(f"Não foi possível encontrar a raiz do projeto contendo '{anchor_file}'.")

# 1. Definir BASE_DIR (Raiz do Projeto)
try:
    BASE_DIR = find_project_root("conf/config.yaml")
    print(f"📂 Raiz do Projeto encontrada: {BASE_DIR}")
except FileNotFoundError as e:
    # Fallback manual caso a busca automática falhe (ajuste se necessário)
    print("Busca automática falhou. Usando fallback.")
    BASE_DIR = Path("/Users/lucasborges/Downloads/TCC")

# 2. Carregar o YAML da pasta conf
CONFIG_PATH = BASE_DIR / "conf/config.yaml"
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# ==============================================================================
# 2. ATALHOS E VARIÁVEIS GLOBAIS
# ==============================================================================

# Atalhos dos Dicionários do YAML
# P['raw'] vai virar algo como: /Users/.../TCC/data/raw
P = {k: BASE_DIR / v for k, v in config['paths'].items()} # P de Paths
F = config['files']                                       # F de Files
PM = config['params']                                     # PM de Params

print(f"⚙️ Configuração carregada de: {CONFIG_PATH}")

# ==============================================================================
# 3. PONTE DE VARIÁVEIS
# ==============================================================================

# Caminhos de Arquivos (Apontando para o YAML)
TRAIN_EMB_PATH       = P['processed'] / F['track_embeddings']
NEW_EMB_PATH         = P['processed'] / F['new_track_embeddings']
X_TRAIN_PATH         = P['processed'] / F['train_features']
X_TEST_PATH          = P['processed'] / F['test_features']

# Se não estiver no YAML, usa o caminho construído:
TRACKS_COMPLETE_PATH = P['interim']   / "df_tracks_complete_v5.parquet"

# Caminhos de Grafos
# Verifica se as chaves existem no yaml, senão usa padrão
MATCHING_MAP_PATH    = P.get('graphs_coarsened', P['graphs_bipartite']) / F['matching_map']
SUPER_EMB_PATH       = P.get('graphs_super', P['graphs_bipartite'])     / F['super_embeddings']

# Parâmetros
SEED                 = PM['seed']

# Configurações Visuais Padrão
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300

📂 Raiz do Projeto encontrada: /Users/lucasborges/Downloads/TCC
⚙️ Configuração carregada de: /Users/lucasborges/Downloads/TCC/conf/config.yaml


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F_nn  # <--- CORREÇÃO: Mudamos de F para F_nn
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from torch_geometric.transforms import RandomLinkSplit
from scipy.sparse import load_npz
from pathlib import Path
import gc

# --- CONFIGURAÇÃO DE CAMINHOS DINÂMICOS ---
# Usamos P (Paths) e F (Files) da Célula Mestra
# Certifique-se de que P e F estão definidos (rode a Célula Mestra antes se necessário)

print("Mapeando arquivos para o GraphSAGE...")

# 1. INPUTS
# Features de Treino
X_TRAIN_PATH = P['processed'] / F['train_features']  # Agora F é o dicionário correto

# Grafo de Super-Músicas
SUPER_ADJ_PATH   = P['graphs_super'] / "A_super_tracks_adjacency.npz"
SUPER_INDEX_PATH = P['graphs_super'] / "super_m_index.parquet"

# Mapa de Matching
MATCHING_MAP_PATH = P['graphs_coarsened'] / F['matching_map']

# 2. OUTPUTS
OUT_SUPER_EMB = P['graphs_super'] / F['super_embeddings']
OUT_TRACK_EMB = P['processed'] / F['track_embeddings']

# 3. VALIDAÇÃO
missing = []
for p in [X_TRAIN_PATH, SUPER_ADJ_PATH, SUPER_INDEX_PATH, MATCHING_MAP_PATH]:
    if not p.exists():
        missing.append(p.name)

if missing:
    print(f"ERRO: Arquivos de entrada não encontrados: {missing}")
else:
    print("Todos os arquivos de entrada encontrados.")
    print(f"   Output Super: {OUT_SUPER_EMB.name}")

Mapeando arquivos para o GraphSAGE...
Todos os arquivos de entrada encontrados.
   Output Super: super_embeddings_mean.parquet


In [3]:
def build_supergraph_data(features_path, super_adj_path, super_index_path, matching_map_path):
    print("--- 1. Carregando Estrutura do Grafo ---")
    super_m_index = pd.read_parquet(super_index_path).squeeze()
    if isinstance(super_m_index, pd.DataFrame): super_m_index = super_m_index.iloc[:, 0]
    
    S_super = load_npz(super_adj_path).tocsr()
    print(f"   Nós: {S_super.shape[0]:,} | Arestas: {S_super.nnz:,}")

    print("\n--- 2. Carregando Features e Mapeando ---")
    df_feats_track = pd.read_parquet(features_path)
    matching_df = pd.read_parquet(matching_map_path)
    
    # Padronizar nomes
    col_uri = "original_track_uri" if "original_track_uri" in matching_df.columns else "track_uri"
    df_map = matching_df[[col_uri, "super_track_id"]].rename(columns={col_uri: "track_uri"})
    if "track_uri" not in df_feats_track.columns and "id" in df_feats_track.columns:
        df_feats_track = df_feats_track.rename(columns={"id": "track_uri"})

    # Merge (Inner Join)
    df_merged = df_feats_track.merge(df_map, on="track_uri", how="inner")
    
    # Filtrar apenas colunas numéricas para agregação
    exclude = ["track_uri", "super_track_id", "pid", "release_year"]
    feature_cols = [c for c in df_merged.columns if c not in exclude and df_merged[c].dtype.kind in 'bifc']
    
    print(f"   Agregando {len(feature_cols)} features (Média)...")
    df_super_feats = df_merged.groupby("super_track_id")[feature_cols].mean().reset_index()

    # Alinhar com a ordem da matriz de adjacência
    df_super_index = pd.DataFrame({"super_track_id": super_m_index.values})
    df_super_feats = df_super_index.merge(df_super_feats, on="super_track_id", how="left").fillna(0.0)
    
    # Criar Tensores
    x = torch.from_numpy(df_super_feats[feature_cols].to_numpy().astype("float32"))
    
    S_coo = S_super.tocoo()
    edge_index = torch.stack([torch.from_numpy(S_coo.row), torch.from_numpy(S_coo.col)], dim=0).long()

    data = Data(x=x, edge_index=edge_index)
    print("\nData Object Criado:", data)
    
    return data, df_super_feats

# Execução
data_super, df_super_feats = build_supergraph_data(X_TRAIN_PATH, SUPER_ADJ_PATH, SUPER_INDEX_PATH, MATCHING_MAP_PATH)

--- 1. Carregando Estrutura do Grafo ---
   Nós: 20,641 | Arestas: 1,032,050

--- 2. Carregando Features e Mapeando ---
   Agregando 51 features (Média)...

Data Object Criado: Data(x=[20641, 51], edge_index=[2, 1032050])


In [4]:
#  Modelo SuperGraphSAGE


class SuperGraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels=64, out_channels=64, 
                 num_layers=2, dropout=0.2, aggr='mean'):
        super().__init__()
        self.dropout_rate = dropout
        self.convs = nn.ModuleList()
        
        # Camada 1
        self.convs.append(SAGEConv(in_channels, hidden_channels, aggr=aggr))
        
        # Camadas Intermediárias
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels, aggr=aggr))
        
        # Camada Saída
        if num_layers > 1:
            self.convs.append(SAGEConv(hidden_channels, out_channels, aggr=aggr))
    
    def forward(self, x, edge_index):
        # 1. Processamento das camadas iniciais e intermediárias
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F_nn.relu(x)
            x = F_nn.dropout(x, p=self.dropout_rate, training=self.training)
        
        # 2. Última camada de convolução
        x = self.convs[-1](x, edge_index)
        
        # 3. NORMALIZAÇÃO L2 DOS VETORES
        # Garante que todos os vetores tenham norma = 1
        # Transformando o produto escalar em Similaridade de Cosseno
        x = F_nn.normalize(x, p=2, dim=-1)
        
        return x


In [5]:
# Funções auxiliares para treinamento e avaliação

import random

def set_seed(seed=42):
    """Trava todas as sementes aleatórias para resultados reproduzíveis."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f"Seed fixada em {seed}")


def bpr_loss(pos_scores, neg_scores):
    """
    Bayesian Personalized Ranking Loss.
    Otimiza para que pos_scores > neg_scores (ordem relativa).
    """
    return -F_nn.logsigmoid(pos_scores - neg_scores).mean()


def compute_validation_metrics(model, val_data, device, num_nodes, sample_size=10000):
    """
    Calcula métricas de validação de forma eficiente.
    Retorna apenas Ranking Accuracy e Average Margin (sem MRR que causa crash).
    """
    model.eval()
    with torch.no_grad():
        z_val = model(val_data.x, val_data.edge_index)
        
        # Arestas positivas (com sampling)
        val_pos_all = val_data.edge_label_index[:, val_data.edge_label == 1]
        
        if val_pos_all.size(1) > sample_size:
            sample_indices = torch.randperm(val_pos_all.size(1))[:sample_size]
            val_pos = val_pos_all[:, sample_indices]
        else:
            val_pos = val_pos_all
        
        # Arestas negativas (random)
        val_neg = torch.randint(0, num_nodes, val_pos.shape, device=device)
        
        # Scores
        pos_scores = (z_val[val_pos[0]] * z_val[val_pos[1]]).sum(dim=-1)
        neg_scores = (z_val[val_neg[0]] * z_val[val_neg[1]]).sum(dim=-1)
        
        # Métricas simples (sem criar matrizes grandes)
        ranking_acc = (pos_scores > neg_scores).float().mean().item()
        avg_margin = (pos_scores - neg_scores).mean().item()
    
    return {'ranking_acc': ranking_acc, 'avg_margin': avg_margin}

In [6]:
# Função principal de treinamento (simplificada)

def train_model(data, hidden_channels=64, epochs=200, lr=0.001, 
                validation_interval=50, seed=42, save_logs=True):
    """
    Treina o modelo SuperGraphSAGE com BPR Loss.
    Versão simplificada sem hard negatives para evitar crash de memória.
    """
    set_seed(seed)
    
    print(f"\n{'='*60}")
    print(f"TREINAMENTO GraphSAGE")
    print(f"Epochs: {epochs} | Hidden: {hidden_channels} | LR: {lr}")
    print(f"{'='*60}\n")
    
    # Split Treino/Validação
    transform = RandomLinkSplit(
        num_val=0.1, num_test=0.0, 
        is_undirected=False, add_negative_train_samples=False
    )
    train_data, val_data, _ = transform(data)
    
    # Setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}\n")
    
    model = SuperGraphSAGE(
        in_channels=data.num_features, 
        hidden_channels=hidden_channels, 
        out_channels=hidden_channels
    ).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_data = train_data.to(device)
    val_data = val_data.to(device)
    
    training_logs = []
    
    # Loop de Treinamento
    for epoch in range(1, epochs + 1):
        model.train()
        optimizer.zero_grad()
        
        z = model(train_data.x, train_data.edge_index)
        
        # Arestas positivas e negativas (random)
        pos_edge = train_data.edge_label_index[:, train_data.edge_label == 1]
        neg_edge = torch.randint(0, data.num_nodes, pos_edge.shape, device=device)
        
        # Scores e Loss
        pos_scores = (z[pos_edge[0]] * z[pos_edge[1]]).sum(dim=-1)
        neg_scores = (z[neg_edge[0]] * z[neg_edge[1]]).sum(dim=-1)
        loss = bpr_loss(pos_scores, neg_scores)
        
        loss.backward()
        optimizer.step()
        
        # Validação periódica
        if epoch % validation_interval == 0:
            metrics = compute_validation_metrics(model, val_data, device, data.num_nodes)
            training_logs.append({
                'epoch': epoch, 'loss': loss.item(),
                'ranking_acc': metrics['ranking_acc'], 'avg_margin': metrics['avg_margin']
            })
            print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | "
                  f"Rank Acc: {metrics['ranking_acc']:.4f} | Margin: {metrics['avg_margin']:.4f}")
    
    # Salvar logs
    if save_logs and training_logs:
        logs_df = pd.DataFrame(training_logs)
        logs_path = P['metrics'] / "training_logs.parquet"
        logs_path.parent.mkdir(parents=True, exist_ok=True)
        logs_df.to_parquet(logs_path, index=False)
        print(f"\nLogs salvos em: {logs_path}")
    
    # Embeddings Finais
    print(f"\n{'='*60}")
    print("GERANDO EMBEDDINGS FINAIS")
    print(f"{'='*60}\n")
    
    model.eval()
    with torch.no_grad():
        final_z = model(data.x.to(device), data.edge_index.to(device)).cpu().numpy()
    
    print(f"Shape: {final_z.shape}")
    print(f"Norma média: {np.linalg.norm(final_z, axis=1).mean():.4f} (esperado ~1.0)")
    
    return final_z

In [7]:
# Treinamento com BPR Loss (versão simplificada)

z_final_bpr = train_model(
    data_super,
    hidden_channels=64,
    epochs=200,
    lr=0.001,
    validation_interval=50,
    seed=42
)

Seed fixada em 42

TREINAMENTO GraphSAGE
Epochs: 200 | Hidden: 64 | LR: 0.001

Device: cpu

Epoch 050 | Loss: 0.4512 | Rank Acc: 0.8676 | Margin: 0.7402
Epoch 100 | Loss: 0.4242 | Rank Acc: 0.9035 | Margin: 0.7754
Epoch 150 | Loss: 0.4109 | Rank Acc: 0.9204 | Margin: 0.7804
Epoch 200 | Loss: 0.4028 | Rank Acc: 0.9327 | Margin: 0.7996

Logs salvos em: /Users/lucasborges/Downloads/TCC/reports/metrics/training_logs.parquet

GERANDO EMBEDDINGS FINAIS

Shape: (20641, 64)
Norma média: 1.0000 (esperado ~1.0)


In [8]:
import gc

# Liberar memória do treinamento
del data_super
gc.collect()

print("\n--- 1. Salvando Embeddings das Super-Músicas ---")
emb_cols = [f"emb_mean_{i:03d}" for i in range(z_final_bpr.shape[1])]

df_super_emb = df_super_feats[["super_track_id"]].copy()
df_super_emb[emb_cols] = z_final_bpr

print(f"   Salvando em: {OUT_SUPER_EMB}")
df_super_emb.to_parquet(OUT_SUPER_EMB, index=False)


print("\n--- 2. Projetando Embeddings para o Treino ---")

# Carregar Mapa (Track -> Super)
matching_df = pd.read_parquet(MATCHING_MAP_PATH)
col_uri = "original_track_uri" if "original_track_uri" in matching_df.columns else "track_uri"
matching_df = matching_df.rename(columns={col_uri: "track_uri"})

# Carregar IDs de Treino
df_train = pd.read_parquet(X_TRAIN_PATH)
if "track_uri" not in df_train.columns and "id" in df_train.columns:
    df_train.rename(columns={"id": "track_uri"}, inplace=True)

# Merge: Track -> Super -> Embedding
df_train_emb = df_train[["track_uri"]].merge(matching_df, on="track_uri", how="inner")
df_train_emb = df_train_emb.merge(df_super_emb, on="super_track_id", how="inner")

if len(df_train_emb) == 0:
    print("ERRO: DataFrame vazio após merge!")
else:
    print(f"   {len(df_train_emb):,} faixas mapeadas.")
    print(f"   Salvando em: {OUT_TRACK_EMB}")
    df_train_emb.to_parquet(OUT_TRACK_EMB, index=False)

print("\n" + "="*50)
print("PIPELINE GRAPHSAGE CONCLUIDO!")
print("="*50)


--- 1. Salvando Embeddings das Super-Músicas ---
   Salvando em: /Users/lucasborges/Downloads/TCC/graphs/super_item_item/super_embeddings_mean.parquet

--- 2. Projetando Embeddings para o Treino ---
   321,160 faixas mapeadas.
   Salvando em: /Users/lucasborges/Downloads/TCC/data/processed/track_embeddings_mean.parquet

PIPELINE GRAPHSAGE CONCLUIDO!
